Step 2 - Database Engineering

Use SQLAlchemy to model your table schemas and create a sqlite database for your tables. You will need one table for measurements and one for stations.

Create a Jupyter Notebook called database_engineering.ipynb and use this to complete all of your Database Engineering work.

Use Pandas to read your cleaned measurements and stations CSV data.

Use the engine and connection string to create a database called hawaii.sqlite.

Use declarative_base and create ORM classes for each table.

You will need a class for Measurement and for Station.

Make sure to define your primary keys.

Once you have your ORM classes defined, create the tables in the database using create_all.

In [23]:
# Dependencies and boilerplate
import pandas as pd
from sqlalchemy import Column, Float, Integer, String
from sqlalchemy.ext.declarative import declarative_base
import sqlalchemy
Base = declarative_base()

In [24]:
#  Read cleaned measurements and stations CSV data
meas_data = pd.read_csv("clean_hawaii_measurements.csv")
sta_data = pd.read_csv("Resources/hawaii_stations.csv")

In [25]:
meas_data.head()

,station,date,prcp,tobs
0,USC00519397,2010-01-01,0.08,65
1,USC00519397,2010-01-02,0.00,63
2,USC00519397,2010-01-03,0.00,74
3,USC00519397,2010-01-04,0.00,76
4,USC00519397,2010-01-06,0.00,73


In [26]:
sta_data.head()

,station,name,latitude,longitude,elevation
0,USC00519397,"WAIKIKI 717.2, HI US",21.2716,-157.8168,3.0
1,USC00513117,"KANEOHE 838.1, HI US",21.4234,-157.8015,14.6
2,USC00514830,"KUALOA RANCH HEADQUARTERS 886.9, HI US",21.5213,-157.8374,7.0
3,USC00517948,"PEARL CITY, HI US",21.3934,-157.9751,11.9
4,USC00518838,"UPPER WAHIAWA 874.3, HI US",21.4992,-158.0111,306.6


In [29]:
# Save the cleaned data to a file called `customers_cleaned.csv`
new_csv = "clean_hawaii_measurements.csv"
new_csv1 = "hawaii_stations.csv"
meas_data.to_csv(new_csv, index=False)

In [31]:
# # Use a Session to test the ORM classes for each table
from sqlalchemy import create_engine, MetaData
from sqlalchemy.orm import Session

# session = Session(bind=engine)
# Create an engine to a database file called "hawaii.sqlite"
engine = create_engine('sqlite:///hawaii.sqlite')

In [32]:
# Create a connection to the engine called `conn`
conn = engine.connect()

In [33]:
# Creates Classes which will serve as the anchor points for our Tables
class Measurement(Base):
    __tablename__ = 'measurement'
    id = Column(Integer, primary_key=True)
    station = Column(String)
    date = Column(String)
    prcp = Column(Float)
    tobs = Column(Integer)


class Station(Base):
    __tablename__ = 'station'
    id = Column(Integer, primary_key=True)
    station = Column(String)
    name = Column(String)
    latitude = Column(Float)
    longitude = Column(Float)
    elevation = Column(Float)



In [34]:
# Use `create_all` to create the customers table in the database
Base.metadata.create_all(engine)

In [35]:
# Load the cleaned csv file into a pandas dataframe
new_df = pd.read_csv(new_csv)
new_df1 = pd.read_csv(new_csv1)

In [36]:
# Use Orient='records' to create a list of data to write
# to_dict() cleans out DataFrame metadata as well
data = new_df.to_dict(orient='records')
data1 = new_df1.to_dict(orient='records')

In [37]:
# Data is just a list of dictionaries that represent each row of data
print(data[:5])

[{'station': 'USC00519397', 'date': '2010-01-01', 'prcp': 0.08, 'tobs': 65}, {'station': 'USC00519397', 'date': '2010-01-02', 'prcp': 0.0, 'tobs': 63}, {'station': 'USC00519397', 'date': '2010-01-03', 'prcp': 0.0, 'tobs': 74}, {'station': 'USC00519397', 'date': '2010-01-04', 'prcp': 0.0, 'tobs': 76}, {'station': 'USC00519397', 'date': '2010-01-06', 'prcp': 0.0, 'tobs': 73}]


In [38]:
# Data is just a list of dictionaries that represent each row of data
print(data1[:5])

[{'station': 'USC00519397', 'name': 'WAIKIKI 717.2, HI US', 'latitude': 21.2716, 'longitude': -157.8168, 'elevation': 3.0}, {'station': 'USC00513117', 'name': 'KANEOHE 838.1, HI US', 'latitude': 21.4234, 'longitude': -157.8015, 'elevation': 14.6}, {'station': 'USC00514830', 'name': 'KUALOA RANCH HEADQUARTERS 886.9, HI US', 'latitude': 21.5213, 'longitude': -157.8374, 'elevation': 7.0}, {'station': 'USC00517948', 'name': 'PEARL CITY, HI US', 'latitude': 21.3934, 'longitude': -157.9751, 'elevation': 11.9}, {'station': 'USC00518838', 'name': 'UPPER WAHIAWA 874.3, HI US', 'latitude': 21.4992, 'longitude': -158.0111, 'elevation': 306.6}]


In [39]:
# Use MetaData from SQLAlchemy to reflect the tables
metadata = MetaData(bind=engine)
metadata.reflect()

In [40]:
# Save the reference to the `customers` table as a variable called `table`
table = sqlalchemy.Table('measurement', metadata, autoload=True)
table1 = sqlalchemy.Table('station', metadata, autoload=True)

In [54]:
# Use `table.delete()` to remove any pre-existing data.
# Note that this is a convenience function so that you can re-run the example code multiple times.
# You would not likely do this step in production.
conn.execute(table.delete())
conn.execute(table1.delete())

In [55]:
# Use `table.insert()` to insert the data into the table
# The SQL table is populated during this step
conn.execute(table.insert(), data)
conn.execute(table1.insert(), data1)

In [46]:
# Test that the insert works by fetching the first 5 rows. 
conn.execute("select * from measurement limit 5").fetchall()

[(1, 'USC00519397', '2010-01-01', 0.08, 65),
 (2, 'USC00519397', '2010-01-02', 0.0, 63),
 (3, 'USC00519397', '2010-01-03', 0.0, 74),
 (4, 'USC00519397', '2010-01-04', 0.0, 76),
 (5, 'USC00519397', '2010-01-06', 0.0, 73)]

In [56]:
# Test that the insert works by fetching the first 5 rows. 
conn.execute("select * from station limit 5").fetchall()

[(1, 'USC00519397', 'WAIKIKI 717.2, HI US', 21.2716, -157.8168, 3.0),
 (2, 'USC00513117', 'KANEOHE 838.1, HI US', 21.4234, -157.8015, 14.6),
 (3, 'USC00514830', 'KUALOA RANCH HEADQUARTERS 886.9, HI US', 21.5213, -157.8374, 7.0),
 (4, 'USC00517948', 'PEARL CITY, HI US', 21.3934, -157.9751, 11.9),
 (5, 'USC00518838', 'UPPER WAHIAWA 874.3, HI US', 21.4992, -158.0111, 306.6)]